***A2 LOADING***

This section requires your neo4j to be closed. We will generate the necessary csvs using a script and then load them into neo4j with a command line. Follow the steps of the notebook carefully.

In [6]:
%run scripts/make.py



CSVs successfully created


In [ ]:
import platform
from dotenv import load_dotenv
import os
import subprocess

load_dotenv()
csv_path = os.getcwd().replace("\\", "/") + "/csv"

DB_PATH= os.environ.get('NEO4J_DB_PATH')


if platform.system() == "Windows":
    print("Copy and paste the following command in your terminal to import the csv files into Neo4j:")
    print("")
    print(f'{DB_PATH}\\bin\\neo4j-admin database import full --overwrite-destination --verbose --delimiter "|" --array-delimiter "^" --nodes=paper="{csv_path}/paper.csv" --nodes=author="{csv_path}/author.csv" --nodes=keyword="{csv_path}/keywords.csv" --nodes=event="{csv_path}/confws.csv" --nodes=event_edition="{csv_path}/confws_edition.csv" --nodes=journal="{csv_path}/journal.csv" --relationships=published="{csv_path}/paper_journal.csv" --relationships=presented="{csv_path}/paper_confws.csv" --relationships=writes="{csv_path}/author_paper.csv" --relationships=cite="{csv_path}/paper_paper.csv" --relationships=has_keyword="{csv_path}/paper_keywords.csv" --relationships=has_corresponding_author="{csv_path}/paper_corresponding_author.csv" --relationships=has_edition="{csv_path}/confws_edition_confws.csv" --relationships=reviewed_by="{csv_path}/paper_review.csv"')
elif platform.system() == "Darwin":
    print("Running the command csv/neo4j_import.sh")
else:
    print("Script only valid for Windows and MacOS, if Linux look for an equivalent command online.")

Copy and paste the following command in your terminal to import the csv files into Neo4j:

C:\Users\josub\Documents\Neo4j\relate-data\dbmss\dbms-63530268-3389-4bdc-9b02-07e7210fa2f0\bin\neo4j-admin database import full --overwrite-destination --verbose --delimiter "|" --array-delimiter "^" --nodes=paper="c:/Users/josub/Desktop/BDMA/Barcelona/Semantic_Data_Management/Property_Graph/csv/paper.csv" --nodes=author="c:/Users/josub/Desktop/BDMA/Barcelona/Semantic_Data_Management/Property_Graph/csv/author.csv" --nodes=keyword="c:/Users/josub/Desktop/BDMA/Barcelona/Semantic_Data_Management/Property_Graph/csv/keywords.csv" --nodes=event="c:/Users/josub/Desktop/BDMA/Barcelona/Semantic_Data_Management/Property_Graph/csv/confws.csv" --nodes=event_edition="c:/Users/josub/Desktop/BDMA/Barcelona/Semantic_Data_Management/Property_Graph/csv/confws_edition.csv" --nodes=journal="c:/Users/josub/Desktop/BDMA/Barcelona/Semantic_Data_Management/Property_Graph/csv/journal.csv" --relationships=published="c:/Us

Congratulations! If followed correctly, your the data should be loaded in neo4j. If you want to inspect it you can run neo4j and run one of the following queries:

CALL db.schema.visualization()

_or_

MATCH (n) RETURN n;

_PD: Please close neo4j now to continue, because we will have to change the configuration of the db to do the next step._

***A3 EVOLVING***

First of all we will have to change our memory alocation configuration to run the next step. Let's start by locating the neo4j.conf file. If using Neo4j Desktop it can be found at:

...\Neo4j\relate-data\dbmss\[db-name]\conf

Open it and change dbms.memory.heap.max_size to 5GB

dbms.memory.heap.max_size=5G

Then save it, open neo4j again and run the following script.

In [6]:
%run scripts/evolve.py


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated function: `id`.} {position: line: 5, column: 14, offset: 110} for query: '\nMATCH (a:author)\nMATCH (af:affiliation)\nWITH a, COLLECT(af) as affs, SIZE(COLLECT(af)) as total\nWITH a, affs[ID(a) % total] as m\nMERGE (a)-[:affiliated_to]->(m)\n'
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated function: `id`.} {position: line: 4, column: 10, offset: 62} for query: '\nMATCH (n:journal|event)\nSET n.reviewerNumber = CASE\n    WHEN ID(n)%3 = 0 THEN 1 \n    WHEN ID(n)%3 = 1 THEN 3\n    ELSE 5\nEND\n'
Received no

If everything was done correctly your db should have changed. To check the changes you can use the same commands as before. 